# CrewAI

Ten notatnik pozwoli Ci zapoznać się z systemami wielo-ageentowymi AI. W trakcie ćwiczenia będziemy wykorzystywać framework [crewAI](https://www.crewai.com/), który omawaliśmy na zajęciach.

Po uzupełnieniu tego notatnika powinieneś/powinnaś wiedzieć:
- jak tworzyć crew i agentów
- jak tworzyć zadania i zlecać ich wykonanie crew
- jak tworzyć systemy sekwencyjne i hierarchiczne

Do wykonania ćwiczeń będą nam potrzebne następujace biblioteki:
```
crewai crewai_tools
```
Dodatkowo, będziemy korzystać z openrouter, by wysyłać zapytania do LLMa. Do tego potrzebujemy API key, a także API base (już podane) i nazwy modelu językowego.
Nazwa modelu powinna być w wpisana następującej formie: "openrounter/{nazwa_modelu}".

In [ ]:
!pip install crewai crewai_tools

In [ ]:
import os

API_KEY = "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"
API_BASE = "https://openrouter.ai/api/v1"
MODEL_NAME = "openrouter/openai/gpt-4o-mini"


os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = API_BASE

Gdy już wybraliśmy model, pora na właściwe zadanie -- tworzenie systemu agentowego.

Dzisiaj stworzysz system do pisania eseju na zadany temat.

## Agenci i narzędzia

Zaczniemy od stworzenia agentów. Będziemy potrzebować:
- researcher, który będzie przeszukiwać sieć, by znaleźć istotne informacje
- planner, który zaplanuje schemat eseju na podstawie treści dostraczonych przez researchera
- writer, który napisze esej na podstawie schematu od planera
- citation manager, który zadba o odpowiednie sformatowanie cytowań
- editor, który sprawdzi esej i przekaże ewentualne uwagi


Agenci są definiowani poprzez rolę (`role`), cel (`goal`), backstory (`backstory`) oraz narzędzia (`tools`). Dodatkowo, możemy im pozwolić na delegowanie zadań lub też nie (`allow_delegation`). W założeniu, nie będziemy pozwalać na delegowanie i będziemy ustawiać wartość tej zmiennej na `False`.



Zacznijmy od tworzenia researchera. Do jego obowiązków będzie należało wyszukiwanie infromacji na zadany przez użytkownika temat. Założmy, że na wyjściu powinien dostarczać listę znalezionych źródeł i krótkie podsumowanie znalezionych informacji.

In [ ]:
from crewai import Agent, Task, Crew, LLM, Process

In [ ]:
researcher = Agent(
    role="Content Researcher",
    goal=(
        "Provide in-depth, insightful and facutally accurate "
        "analysis and list of appropriate sources "
        "on the topic: {topic}."
    ),
    backstory=(
        "An experienced researcher "
        "able to provide analysis and useful resources on any given topic "
        "when creating a list of resources researcher might need the help of Citation Manager. "
        "Researcher provides only existing resources and makes sure they are relevant. "
        "Researcher's work is the basis for the Content Planner and Content Writer to "
        "write the outline and the essay."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

Następnie tworzymy plannerai writera.

Planner ma za zadanie zaplanować outline eseju.
Writer na podstawie takiego outline'u ma napisać esej.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan insightful and factually accurate content on {topic}",
    backstory=(
        "An agent that is working on planning an essay "
        "on the topic: {topic}."
        "Content Planner collects information with the help of Content Researcher. "
        "Based on the information, Content Planner writes the outline of the essay. "
        "Content Planner also includes the Researcher's list of sources and relevant information. "
        "Planner's work is the basis for "
        "the Content Writer to write an article on this topic."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate essay on the topic: {topic}",
    backstory=(
        "A agent working on a writing "
        "an essay on the topic: {topic}. "
        "Content Writer bases their writing on the work of "
        "the Content Planner, who provides an outline "
        "and relevant context about the topic. "
        "Content Writer follows the main objectives and "
        "direction of the outline, "
        "provided by the Content Planner. "
        "Content Writer also provides objective and impartial insights "
        "and backs them up with information "
        "provided by the Content Planner. "
        "Content Writer uses evidence and reasoning. "
        "Content Writer makes sure to back-up each argument with a proper source, "
        "provided by Content Planner."
    ),
    verbose=True,
    allow_delegation=False,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

Tworzymy kolejnego agenta: editora. Editor będzie miał za zadanie poprawiać esej dostarczony przez writera.

In [ ]:
editor = Agent(
    role="Editor",
    goal=(
        "Edit a given essay on the topic: {topic} to align with "
        "the writing style of the organization. "
    ),
    backstory=(
        "An agent that is an editor who receives an essay "
        "from the Content Writer. "
        "Editor's goal is to review the essay "
        "to ensure that it follows best practices:"
        "analyzes the topic, defines arguments, uses evidence and reasoning and is written coherently."
        "Editor makes sure that the essay provides balanced viewpoints "
        "and also avoids major controversial topics "
        "or opinions when possible."
        "Editor also checks whether all provided sources actually exist and are used properly."
        "Editor makes sure that the cited sources are cited in the MLA style of citations."
    ),
    allow_delegation=False,
    verbose=True,
    max_iterations=2,
    llm=LLM(
        model=MODEL_NAME,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    ),
)

Ostatnim członkiem naszej załogi będzie citation manager.

**Zadanie 1**

**Stwórz agenta o roli Citation Manager. Jego celem jest sformatowanie cytowań i źródeł w stylu MLA. Opisz odpoiwednio rolę, cel i backstory dla tego agenta. Następnie ustaw zmienne: `allow_delegation=False`, `verbose=True`, `max_iterations=2`. Przypisz agentowi LLMa opartego na gpt-4o-mini (tak jak w przypadku poprzednio tworzonych agentów).**

In [ ]:
citation_manager = Agent(
   ...
)

## Zadania

Teraz tworzymy zadanie dla każdego ze stworzonych agentów.
Każde zadanie zawiera opis, oczekiwany output oraz opcjonalnie agenta, który je wykonuje.

Kolejno definujemy zadania "research", "plan", "write", "edit".

In [ ]:
research = Task(
    description=(
        "1. Conduct comprehensive research on the topic: {topic}.\n"
        "2. Prioritize identifying the latest trends, impactful arguments, and noteworthy news related to {topic}.\n"
        "3. Curate a list of relevant and reliable resources to support the essay's arguments.\n"
        "4. Organize findings into key categories to assist the Content Planner in developing a structured outline.\n"
        "5. Provide a brief summary for each resource explaining its relevance to the topic.\n"
        "6. Make sure to only provide factually accurate information.\n"
    ),
    expected_output=(
        "A well-organized research document containing:\n"
        "- A categorized summary of key findings.\n"
        "- A list of reliable and relevant resources with explanations.\n"
        "- Insights into recent trends, impactful arguments, and noteworthy news.\n"
    ),
    agent=researcher,
)

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, impactful arguments, "
            "and noteworthy news on {topic}.\n"
        "2. Develop a detailed essay content outline including "
            "an introduction, key points, and a call to action.\n"
        "3. Include SEO keywords and relevant data or sources obtained from the Content Researcher.\n"
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and properly formatted resources.",
    agent=planner,
)

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "essay on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		    "3. Sections/Subtitles are properly named "
            "in an academical manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Make sure that each presented argument is supported by a proper citation.\n"
        "6. Proofread for grammatical errors and "
            "alignment with the academical style.\n"
        "7. Include a list of used sources used.\n"
    ),
    expected_output="A well-written essay "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs. At the end, there should be a list of sources used. "
        "The answer should include only the essay and list of sources, without any additional information.",
    agent=writer,
)

In [ ]:
edit = Task(
    description=(
        "1. Review the essay on the topic: {topic} for coherence, readability, and alignment with the organization's writing style.\n"
        "2. Ensure the essay:\n"
        "   - Analyzes the topic thoroughly and defines arguments clearly.\n"
        "   - Uses evidence and reasoning to support each argument.\n"
        "   - Avoids major controversial topics or opinions unless explicitly required.\n"
        "   - Provides balanced viewpoints where applicable.\n"
        "3. Verify the existence and relevance of all cited sources.\n"
        "4. Check whether all sources are cited in proper MLA format.\n"
        "5. Proofread the essay for grammatical accuracy, stylistic consistency, and adherence to best practices in essay writing.\n"
        "6. Ensure the final content is engaging, error-free, and ready for publication.\n"
    ),
    expected_output=(
        "A polished and edited essay with:\n"
        "- Improved coherence and alignment with the organization's writing style.\n"
        "- Verified and properly formatted MLA citations.\n"
        "- Balanced arguments supported by evidence and reasoning.\n"
        "- Grammatical and stylistic accuracy.\n"
        "The output should only include the essay and list of resource, without any additional information."
    ),
    agent=editor,
)

**Zadanie 2**

**Napisz zadanie manage_citations. Zadanie powinno polegać na: po pierwsze, zweryfikowaniu źródeł z eseju, sformatowaniu ich w odpowiedni sposób stosując format MLA, dobrym zorganizowaniu sformatowanych źródeł. Efektem zadania powinna być własnie taka odpowiednio sformatowana lista i poprawnie sformatowane cytowania w tekście. Przypisz do zadania agenta `citation_manager`.**

In [ ]:
manage_citations = Task(
    ...
)

## Tworzenie crew

Mając już określonych agentów oraz zadania musimy skonstrkuować crew.

**Zadanie 3**

**Stwórz crew do pisania eseju uzupełniając parametry `agents` i `tasks`. Te parametry przyjmują listy odpowiednio agentów lub tasków. Przy taskach, wypisz zadania w odpowiedniej kolejności, tak aby proces przebiegł prawidłowo. Dodatkowo ustaw zmiennę `verbose=True`**


In [ ]:
crew = Crew(
    agents=...,
    tasks=...,
    verbose=True,
    process=Process.sequential
)

## Pisanie eseju

Teraz zobaczymy jak nasz system radzi sobie z zadaniem.

**Zadanie 4**

**Wybierz dowolny temat eseju.**

In [ ]:
essay_topic = ...

result = crew.kickoff(inputs={"topic": essay_topic})

Zobaczmy teraz jak wygląda nasz esej.

In [ ]:
from IPython.display import Markdown
Markdown(str(result))

## Procesy hierarchiczne

Póki co korzystaliśmy tylko z procesu sekwencyjnego i dokładnie musieliśmy określić każde z zadań. Teraz spróbujemy użyć dodatkowego agenta -- managera, który będzie zarządzał pozostałymi agentami, w celu napisania eseju.

Nie będziemy tutaj musieli tak dokładnie rozpisywać zadań. Określimy tylko ogólne zadanie "pisanie eseju", a resztą zajmie się manager.

**Możesz dostosować opis managera, żeby był bardziej dopasowany do konkretnego zadania pisania eseju.**

In [ ]:
MODEL_NAME_MANAGER = 'openrouter/openai/gpt-3.5-turbo'

In [ ]:
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion by delegating the tasks. ",
    backstory=(
        "You are an experienced project manager, "
        "skilled in overseeing complex projects and "
        "guiding teams to success. "
        "Your role is to coordinate the efforts of all of the crew members, "
        "ensuring that each task is completed to the highest standard."
        "You should use the expertise of the other crew members to correctly perform smaller tasks.  "
        "You are very busy and should not perform any task on your own, just focus on proper delegation."
    ),
    allow_delegation=True,
    verbose=True,
    max_iterations=15,
    llm=LLM(
        model=MODEL_NAME_MANAGER,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=API_BASE,
        temperature=0.7,
    )
)

In [ ]:
task = Task(
    description=(
        "Write an essay on the given topic: {topic}. "
        "The essay should adhere to the rules of writing a good essay. "
        "The essay should be well-writted and have a clear structure. "
        "Each section of the essay should have 2 to 3 paragraphs. "
        "Each argument presented in the essay should be supported by a proper and relevant citation. "
        "The citations should be fatually accurate and the sources should exist. "
        "At the end of the essay, there should be a list of used sources in the MLA citation style "
        "with the help of Citation Manager. "
        "The essay should be verified by the Editor before final submission. "
    ),
    expected_output=(
        "A well-written essay "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs. In the end, there should be a list of sources used. "
        "The output should only include the essay and list of resource, without any additional information. "
    ),
)

Jak widać, w taki sposób możemy zaoszczędzić dużo wysiłku na pisanie samego procesu. Udało nam się streścić główny zamysł w jednym zadaniu i liczymy na to, że resztą zajmie się manager.

Pora na zdefiniowanie crew. Tutaj właśnie użyjemy procesu hierarchicznego.

**Zadanie 5**

**Stwórz crew do pisania eseju, wykorzysując proces hierarchiczny. Pamiętaj, że agent `manager` nie powinien być wpisany w listę agentów -- poszukaj w [dokumentacji crewAI](https://docs.crewai.com/concepts/crews), w jaki sposób go użyć. Crew powinna otrzymać tylko jedengo taska (tego zdefiniowanego powyżej).**

In [ ]:
crew = Crew(
    ...
)

Użyjemy tego samego tematu, co poprzednio.

In [ ]:
result = crew.kickoff(inputs={"topic": essay_topic})

Niewykluczone, że w pewnym momencie zobaczyłeś/zobaczyłaś błąd, informujący o problemie z delegacją zadania lub nasz manager nie wykorzystał pełnego potencjału crew. Należy pamiętać, że mimo wszystko mamy do czynienia z LLMami -- czasami taki model nie dostosuje się do formatu, którego oczekuje crewAI, co potencjalnie doprowadzi do błędu. Dodatkowo, LLM może pójść na skróty lub zgubić się ze własnym rozumowaniu, przez co nie oddeleguje zadań w pełni poprawnie.

In [ ]:
from IPython.display import Markdown
Markdown(str(result))

**Zadanie 6**

**Którym sposobem otrzymaliśmy lepszy esej? Czy eseje mocno różnią się od siebie? Możesz też porównać esej wygenerowany przy pomocy crew z esejem, który otrzymasz używając ChataGPT (najlepiej z gpt-4o-mini).**